In [ ]:
# download nltk stopwords
import nltk
from google.cloud import storage
nltk.download('stopwords')

In [2]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Copy one wikidumps files 
import os
from pathlib import Path

project_id = 'assignment3-370517'

!gcloud config set project {project_id}

bucket_name = 'amit-chen-bucket-1'

postings_gcp_text = 'postings_gcp_text'
postings_gcp_anchor = 'postings_gcp_anchor'
postings_gcp_title = 'postings_gcp_title'
postings_gcp_text = 'postings_gcp_text_stemmed'
postings_gcp_anchor = 'postings_gcp_anchor_stemmed'
postings_gcp_title = 'postings_gcp_title_stemmed'
doc_len = 'dl'
doc_title = 'dt'
nf_body = 'nf'

# uncoment data you want to upload from bucket 

!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_text}/ . 
!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_anchor}/ . 
!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_title}/ .  
!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_text_stemmed}/ . 
!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_anchor_stemmed}/ . 
!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_title_stemmed}/ .  
!gsutil -m cp -r gs://{bucket_name}/{nf_body}/ . 
!gsutil -m cp -r gs://{bucket_name}/{doc_len}/ . 
!gsutil -m cp -r gs://{bucket_name}/{doc_title}/ . 

In [5]:
!mkdir pr/
!mkdir pv/
client = storage.Client()
blobs = client.list_blobs(f"{bucket_name}")
for b in blobs:
    if "part-00000-8b293cd5-fd79-47e7-a641-3d067da0c2b0-c000.csv.gz" in b.name:
      b.download_to_filename("pr/pr.csv.gz")
    if "pageview_pageviews-202108-user.pkl" in b.name:
      b.download_to_filename("pv/pv.pkl")

In [ ]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

In [ ]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 2JsDuuUTH9LwAeBbnQqZXkbbdod_5NLnteyuKr4NCqq1Knyok

In [ ]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

# Unzip training files
upload and run below code:

In [ ]:
!unzip all_files_for_frontend.zip

# Messing Around 🦖

In [64]:
from flask import Flask, request, jsonify, render_template
import gzip
import pickle
import pandas as pd
from inverted_index_gcp import *

INDEX_FILE = "index"
POSTINGS_GCP_TEXT_INDEX_FOLDER_URL = "postings_gcp_text"
POSTINGS_GCP_ANCHOR_INDEX_FOLDER_URL = "postings_gcp_anchor"
POSTINGS_GCP_TITLE_INDEX_FOLDER_URL = "postings_gcp_title"
POSTINGS_GCP_TEXT_STEMMED_INDEX_FOLDER_URL = "postings_gcp_text_stemmed"
POSTINGS_GCP_ANCHOR_STEMMED_INDEX_FOLDER_URL = "postings_gcp_anchor_stemmed"
POSTINGS_GCP_TITLE_STEMMED_INDEX_FOLDER_URL = "postings_gcp_title_stemmed"
PAGE_RANK_URL = "pr/pr.csv.gz"
PAGE_VIEW_URL = "pv/pv.pkl"
DT_PATH = "dt/dt.pkl"
DL_PATH = "dl/dl.pkl"
NF_PATH = "nf/nf.pkl"

# open files (inverted indexes etc...)
inverted_index_body = InvertedIndex.read_index(POSTINGS_GCP_TEXT_INDEX_FOLDER_URL, INDEX_FILE)
inverted_index_anchor = InvertedIndex.read_index(POSTINGS_GCP_ANCHOR_INDEX_FOLDER_URL, INDEX_FILE)
inverted_index_title = InvertedIndex.read_index(POSTINGS_GCP_TITLE_INDEX_FOLDER_URL, INDEX_FILE)
inverted_index_body_stemmed = InvertedIndex.read_index(POSTINGS_GCP_TEXT_STEMMED_INDEX_FOLDER_URL, INDEX_FILE)
inverted_index_anchor_stemmed = InvertedIndex.read_index(POSTINGS_GCP_ANCHOR_STEMMED_INDEX_FOLDER_URL, INDEX_FILE)
inverted_index_title_stemmed = InvertedIndex.read_index(POSTINGS_GCP_TITLE_STEMMED_INDEX_FOLDER_URL, INDEX_FILE)

with open(DL_PATH, 'rb') as f:
    DL = pickle.load(f)
    DL_LEN = len(DL)

with open(DT_PATH, 'rb') as f:
    DT = pickle.load(f)

with open(NF_PATH, 'rb') as f:
    NF = pickle.load(f)

with open(PAGE_VIEW_URL, 'rb') as f:
    page_view = pickle.load(f)

with gzip.open(PAGE_RANK_URL) as f:
    page_rank = pd.read_csv(f, header=None, index_col=0).squeeze("columns").to_dict()
    max_pr_value = max(page_rank.values())
    page_rank = {doc_id: rank/max_pr_value for doc_id, rank in page_rank.items()}

# Utils


In [65]:
import re
import math
from collections import Counter
from nltk.corpus import stopwords

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    Parameters:
    -----------
    text: string , represting the text to tokenize.
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    RE_WORD = re.compile(r"""[\#\@\w](['\-]?[\w,]?[\w.]?(?:['\-]?[\w,]?[\w])){0,24}""", re.UNICODE)
    english_stopwords = frozenset(stopwords.words('english'))
    corpus_stopwords = ["category", "references", "also", "external", "links",
                        "may", "first", "see", "history", "people", "one", "two",
                        "part", "thumb", "including", "second", "following",
                        "many", "however", "would", "became"]

    all_stopwords = english_stopwords.union(corpus_stopwords)

    list_of_tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]
    return list_of_tokens

def BM25(tokens, K, B, AVGDL, inverted_index, index_folder_url, DL, DL_LEN):
    
    doc_BM25_value = Counter()

    for token in tokens:

        # calc idf for specific token
        try:
          token_df = inverted_index.df[token]
        except:
            continue
        token_idf = math.log(DL_LEN/token_df,10)

        # loading posting list with (word, (doc_id, tf))
        posting_list = inverted_index.read_posting_list(token, index_folder_url)
        for page_id, word_freq in posting_list:
            #normalized tf (by the length of document)
            numerator = word_freq*(K+1)
            denominator = word_freq + K*(1-B + (B*DL[page_id])/AVGDL)
            doc_BM25_value[page_id] += token_idf*(numerator/denominator)
        
    sorted_doc_BM25_value = doc_BM25_value.most_common()
    return sorted_doc_BM25_value

def cossim(tokens, inverted_index, index_folder_url, DL, DL_LEN, NF):
    
    # get frequency of each token in query
    query_freq = Counter(tokens)

    numerator = Counter()
    query_denominator = 0
    weight_token_query = 0

    query_len = len(tokens)
    for token in tokens:

        # calc idf for specific token
        try:
          token_df = inverted_index.df[token]
        except:
            continue
        token_idf = math.log(DL_LEN/token_df,10)

        # calc query_token_tf
        tf_of_query_token = query_freq[token]/query_len
        weight_token_query = tf_of_query_token*token_idf
        query_denominator += math.pow(weight_token_query ,2)

        # loading posting list with (word, (doc_id, tf))
        posting_list = inverted_index.read_posting_list(token, index_folder_url)
        for page_id, word_freq in posting_list:
            #normalized tf (by the length of document)
            try:
                tf = (word_freq/DL[page_id])
                weight_word_page = tf*token_idf
                numerator[page_id] += weight_word_page*weight_token_query
            except:
                pass

    cosim = Counter()
    for page_id in numerator.keys():
      cosim[page_id] = numerator[page_id]/((math.sqrt(query_denominator)*NF[page_id]))
    
    sorted_doc_cossim_value = cosim.most_common()
    return sorted_doc_cossim_value


In [66]:
def search(query):
    res = []
    if len(query) == 0:
      return res

    # tokenizing the query
    tokens = tokenize(query)

    # cossim without stemming
    sorted_doc_score_pairs = cossim(tokens, inverted_index_body, POSTINGS_GCP_TEXT_INDEX_FOLDER_URL, DL, DL_LEN, NF)
    
    # BM25
    # K = ?
    # B = ?
    # AVGDL = 341.0890174848911
    # sorted_doc_score_pairs = BM25(query, K, B, AVGDL, inverted_index_body, POSTINGS_GCP_TEXT_INDEX_FOLDER_URL, DL, DL_LEN)

    # take first 100 
    best = sorted_doc_score_pairs[:100]
    print(best)

    # take page titles according to id
    res = [(x[0], DT[x[0]]) for x in best]


def search_body(query):
    res = []
    if len(query) == 0:
      return res
    
    # tokenizing the query
    tokens = tokenize(query)

    # cossim
    sorted_doc_score_pairs = cossim(tokens, inverted_index_body, POSTINGS_GCP_TEXT_INDEX_FOLDER_URL, DL, DL_LEN, NF)
    
    # take first 100 
    best = sorted_doc_score_pairs[:100]
    print(best)

    # take page titles according to id
    res = [(x[0], DT[x[0]]) for x in best]

    return res


def search_title(query):
    res = []
    if len(query) == 0:
      return res
    # tokenizing the query
    tokens = tokenize(query)

    # loading posting list with (word, (doc_id, tf))
    posting_lists = inverted_index_title.get_posting_lists(tokens, POSTINGS_GCP_TITLE_INDEX_FOLDER_URL)

    tf_dict = {}
    for posting in posting_lists:
        for doc_id, tf in posting:
            if doc_id in tf_dict:
                tf_dict[doc_id] += 1
            else:
                tf_dict[doc_id] = 1

    list_of_docs = sorted([(doc_id, score) for doc_id, score in tf_dict.items()], key=lambda x: x[1], reverse=True)
    print(list_of_docs)
    res = [(doc_id, DT[doc_id]) for doc_id, score in list_of_docs]
    return res


def search_anchor(query):
    res = []
    if len(query) == 0:
      return res
    # tokenizing the query
    tokens = tokenize(query)

    # loading posting list with (word, (doc_id, tf))
    posting_lists = inverted_index_anchor.get_posting_lists(tokens, POSTINGS_GCP_ANCHOR_INDEX_FOLDER_URL)
    
    tf_dict = {}
    for posting in posting_lists:
        for doc_id, tf in posting:
            if doc_id in tf_dict:
                tf_dict[doc_id] += 1
            else:
                tf_dict[doc_id] = 1

    list_of_docs = sorted([(doc_id, score) for doc_id, score in tf_dict.items()], key=lambda x: x[1], reverse=True)[:100]
    print(list_of_docs)
    for doc_id, score in list_of_docs:
        try:
            res.append((doc_id, DT[doc_id]))
        except:
            pass
    return res


def get_pagerank(wiki_ids):
    res = []
    if len(wiki_ids) == 0:
      return res

    for wiki_id in wiki_ids:
      try:
        res.append(page_rank[wiki_id])
      except:
        res.append(None)

    return res


def get_pageview(wiki_ids):
    res = []
    if len(wiki_ids) == 0:
      return res
      
    for wiki_id in wiki_ids:
      try:
        res.append(page_rank[wiki_id])
      except:
        res.append(None)

    return res


# Run Quries


In [ ]:
print(search_anchor("information retrieval"))
# print(search_title("information retrieval"))
# print(get_pagerank([123456, 679125, 3978521]))
# print(get_pageview([45687, 778985, 887795]))

# Run App

In [6]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://527f-34-171-74-87.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Jan/2023 10:35:44] "GET /search?query=best+marvel+movie HTTP/1.1" 200 -


[(5027882, 0.7418604775444455), (39632709, 0.7386383876549714), (306960, 0.727912433618154), (1074657, 0.6954088809011663), (5727147, 0.692150715416282), (403579, 0.6704012023885402), (66113901, 0.6702747131550605), (20966, 0.6647393424182793), (8188167, 0.6526015206684286), (17448401, 0.6407888813584854), (15311317, 0.6395489475492644), (35871824, 0.6317080109629694), (17960196, 0.6280988925505265), (26552494, 0.6249622890377099), (33308367, 0.6136749489600388), (55994110, 0.606806116171739), (894151, 0.6044946941877017), (2453882, 0.6021268827554088), (41008758, 0.6016420599892961), (842924, 0.5990730161478861), (1047719, 0.5964203850912859), (44611175, 0.5915988925865847), (49287801, 0.5900108756833728), (5020000, 0.5850338052498854), (60132806, 0.5849072011312106), (42721570, 0.5811697812701522), (2802921, 0.5799848765278884), (66069085, 0.5792783868089857), (58488308, 0.577647024263518), (9728333, 0.5774331653897649), (10181183, 0.5758524365825953), (17586817, 0.5689786462985459),

INFO:werkzeug:127.0.0.1 - - [13/Jan/2023 10:35:48] "GET /search?query=How+do+kids+come+to+world%3F HTTP/1.1" 200 -


...

[(1861944, 0.645210979915685), (2347978, 0.5177365555704602), (6902155, 0.4930020242635897), (5617195, 0.4781133687572856), (4789487, 0.4658955156341635), (7081866, 0.46288553297333157), (538871, 0.4580292765779094), (10905655, 0.438307098866161), (2250249, 0.4315462618232814), (9860955, 0.42689773401540687), (6680309, 0.4244035513509471), (2403481, 0.4225335665490819), (810230, 0.4212820354967698), (6236356, 0.41844175689488977), (7317283, 0.4184001538012158), (5681949, 0.4095901656269586), (6873605, 0.4057795419505675), (4151001, 0.40217148397123054), (5221695, 0.40155257298160024), (172707, 0.39998708040251413), (4118257, 0.3956071093415681), (978461, 0.39447684613539613), (8010446, 0.3937978105125669), (7758365, 0.3921622778453916), (1668452, 0.3862698671730832), (2065009, 0.38316243434743635), (4634538, 0.38308209787824027), (2573205, 0.38306189181855554), (7528266, 0.3776267274576423), (5857965, 0.37255034748113824), (12259587, 0.371842324792009), (2493224, 0.36891711395327975), 

INFO:werkzeug:127.0.0.1 - - [13/Jan/2023 10:36:19] "GET /search?query=Elon+musk HTTP/1.1" 200 -


[(41309419, 0.7646797367763), (43407192, 0.7598646256681968), (66430882, 0.7193937299058891), (50399439, 0.7173041328786111), (803102, 0.6847561965686744), (65212863, 0.6812022910581487), (48778030, 0.6739679848726008), (65175052, 0.660447383994081), (4666466, 0.6412584201264464), (909036, 0.636831432373363), (33986468, 0.625045276232102), (22248960, 0.6182486150027602), (220806, 0.6166157276584527), (29414816, 0.6105698615512644), (58173986, 0.6082444976045213), (127628, 0.5958248428616006), (47190535, 0.5886617105267613), (21523886, 0.573477895137428), (68289769, 0.5734283470243533), (1001122, 0.570716826777269), (20339194, 0.5524507923660585), (41318207, 0.5463486470062628), (21450611, 0.5446081981485498), (45186981, 0.5409909509652617), (60955975, 0.539076591511531), (32151967, 0.5360295533520133), (6973013, 0.5354259821250418), (58212588, 0.534838738516725), (4555268, 0.5305579494112737), (61490363, 0.5290911181554261), (48703999, 0.5241650607796684), (5399134, 0.5202516165556192)

INFO:werkzeug:127.0.0.1 - - [13/Jan/2023 10:36:20] "GET /search?query=How+do+you+breed+flowers%3F HTTP/1.1" 200 -


[(9507921, 0.7072223773176642), (39676695, 0.6931306004992936), (801773, 0.6669144095164994), (895650, 0.658343329569042), (13780186, 0.6510321689725893), (20897928, 0.6446115467224508), (15446490, 0.6269068769762832), (44066718, 0.6259695015147796), (60649381, 0.6170155448043436), (51118248, 0.5818676638384633), (64576596, 0.5763163004891468), (3550726, 0.5685086220197231), (32908252, 0.5608336937392411), (24016757, 0.5605059318141773), (59673454, 0.5591305244944705), (17843999, 0.5539164484612574), (30510881, 0.5498781410675636), (37570236, 0.547173132660456), (5116464, 0.5420410538421726), (921491, 0.5103395854033316), (5861432, 0.5063389283043987), (6947021, 0.5027451246884785), (1810335, 0.5020757359856827), (37260437, 0.4970848739469803), (21957462, 0.4968248786070023), (62781095, 0.49620880078404084), (20041575, 0.4933006639945657), (7525682, 0.49024362518720915), (798084, 0.48302323111916756), (4914967, 0.4812593134370859), (43426910, 0.4779331376123281), (39588381, 0.475614715

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://cded-35-192-160-143.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_body', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))
print(qs_res)